# CNET 5442 — Sports Analytics Through Data and Networks (Spring 2026)  
## Class 05 — Regression Pt. 1 — Moneyball Replication

## Homework 1, question 4

![](images/moneyball_screenshot.jpg)

__________
## Next time...
Regression Pt. 2 — Expectation & Likelihood `class_06_regression_02_expectation_likelihood.ipynb`
_______

## References and further resources:

1. Class Webpages
    - Github: https://github.com/jkbren/cnet5442_sp26
    - Syllabus and course details: https://brennanklein.com/cnet5442-spring26
2. 